# WMDSP - Exact algorithm for finding the minimum dominating set in weigthed graphs

$$
\begin{aligned}
& \underset{x}{\text{minimize}}
& & \sum_{i=1}^{n} x_i \ \\
& \text{subject to}
&& \sum_{j=1}^{n} weigths_{i,j} \cdot x_j \geq k \cdot (1 - x_i), \quad i=1,\ldots,n \ \\
& & & x_i \in \{0,1\}, \quad i=1,\ldots,n
\end{aligned}
% & & \sum_{j=1}^{n} AdjMatrix_{i,j} \cdot x_j > 0, \quad i=1,\ldots,n \ \\

$$

Given an undirected graph G = (V, E) with vertex set V and edge set E, where each edge e in E has a weight w(e), the Dominating Set with Weighted Edges problem is to find a subset S of V such that:

1. For every vertex v in V, either v is in S or v has a neighbor in S. In other words, every vertex in V is either in S or adjacent to a vertex in S. This property is known as domination.

2. The sum of the weights of the edges in E between vertices in V that are not in S and vertices in S is at least k. In other words, the sum of w(e) over all edges e between vertices in V\S and vertices in S is k. This property takes into account the weights of the edges between V\ S and S.

The goal of the Dominating Set with Weighted Edges problem is to find the smallest subset S of V that satisfies the above properties.

# Importación de librerias

In [1]:
import os
import gurobipy as gp

# Definición de funciones

In [2]:
def create_graph(filename):
    with open('../src/resources/graphs/random/' + filename) as f:
        lines = f.readlines()
    
    weights = [[float(x) for x in line.split()] for line in lines]
    n = len(weights)

    # Construct the edge set and weight dictionary
    V = list(range(0, n))
    E = {}
    W = {}
    for i in range(n):
        for j in range(i, n):
            if weights[i][j] > 0:
                E.setdefault(i, []).append(j)
                E.setdefault(j, []).append(i)
                W[i, j] = weights[i][j]
                W[j, i] = weights[i][j]

    return V, E, W

In [3]:
def find_dominating_set(V, E, W, K, time_limit):
    m = gp.Model("Weigthed MDSP")

    # Create variables
    x = m.addVars(V, vtype=gp.GRB.BINARY, name='x')

    # Set objective
    m.setObjective(x.sum(), sense=gp.GRB.MINIMIZE)

    # Add constraints
    for u in V:
        m.addConstr(gp.quicksum(x[v]*W[u,v] for v in E[u]) >= K*(1 - x[u]))

    # disable gurobi output
    m.setParam('OutputFlag', 0)
    
    # Set time limit
    m.setParam('TimeLimit', time_limit)
    
    # Optimize model
    m.optimize()
    
    solution = []
    for v in V:
        if x[v].x > 0.5:
            solution.append(v)

    return len(solution), m.Runtime, m.status == gp.GRB.OPTIMAL

In [4]:
def execute_wmdsp(graph, K, time_limit):
    V, E, W = create_graph(graph)
    size, runtime, optimal = find_dominating_set(V, E, W, K, time_limit)
    result = [size, K, round(runtime, 2), optimal]
    return result

# Definición de parámetros y ejecución

In [ ]:
graphs = []
for filename in os.listdir('../src/resources/graphs/random/'):
    if filename.endswith('.txt'):
        graphs.append(filename)

# set the time limit for each graph in seconds (1800 = 30 minutes)
time_limit = 1800

# set the values of K
K = [0.1, 0.25, 0.5, 0.75, 1]

with open('../src/resources/results/random-gurobi.csv', 'w') as f:
    f.write('graphName,K,size,runtime(s),optimal?\n')
    for graph in graphs:
            for k in K:
                result = execute_wmdsp(graph, k, time_limit)
                f.write(graph + ',' + str(k) + ',' + str(result[0]) + ',' + str(result[2]) + ',' + str(result[3]) + '\n')